# 浙江地区森林生长监测
利用 *Modis MCD12Q1* 地物分类数据产品和 *MODIS MOD13Q1* 16天标准植被指数产品，对浙江省林区进行监测。

## 初始化环境

In [1]:
import aie
aie.Authenticate()
aie.Initialize()

计算资源初始化中，请等待...
计算资源初始化完成.


## 定义矢量区域
用户可根据自主上传的矢量边界，定义检索数据的区域

In [2]:
region = aie.Geometry({"type":"MultiPolygon","coordinates":[[[[117.775957,26.921058],[122.457225,27.074256],[122.513643,31.387192],[117.521295,31.305655],[117.775957,26.921058]]]],"bbox":[117.521295,26.921058,122.513643,31.387192]})

        
map = aie.Map(
    center=region.getCenter(),
    height=800,
    zoom=5
)

vis_params = {
    'color': '#00FF00'
}
map.addLayer(
    region,
    vis_params,
    'shapefile',
    bounds=region.getBounds()
)
map


Map(center=[29.154125, 120.017469], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## 地物分类数据检索
 使用 *Modis MCD12Q1* 地物分类数据产品

In [3]:
# map
LC_Dataset = aie.ImageCollection('MODIS_MCD12Q1_006') \
                .filterDate('2020-05-01', '2020-05-31')

img = LC_Dataset.select(['LC_Type1']).mean()

forest = img.lt(aie.Image.constant(6))  #.rename('forest')   #1-5为不同类型的森林


vis_params = {
    'bands': 'LC_Type1',
    'min': 1,
    'max': 17,
    'palette': [ 
        '#05450a', '#086a10', '#54a708', '#78d203', '#009900', '#c6b044', 
        '#dcd159', '#dade48', '#fbff13', '#b6ff05', '#27ff87', '#c24f44', 
        '#a5a5a5', '#ff6d4c', '#69fff8', '#f9ffa4', '#1c0dff'
    ]
}

forest_vis = {
    'bands': 'LC_Type1',
    'min': 0,
    'max': 1,
    'palette': [ 
       '#ffffff','#4fb104'
    ]
}

map.addLayer(
    img,
    vis_params,
    'LC_data',
    bounds=region.getBounds()
)

map.addLayer(
    forest,
    forest_vis,
    'Forest',
    bounds=region.getBounds()
)
map




Map(center=[29.154125, 120.017469], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## NDVI数据产品
使用 *MODIS MOD13Q1* 16天标准植被指数产品

In [4]:
# 指定检索数据集，可设置检索的时间范围
def MOD13Q_NDVI(start_time, end_time):
    VI_Dataset = aie.ImageCollection('MODIS_MOD13Q1_006') \
                 .filterDate(start_time, end_time)\
                 .select(['NDVI'])\
                 .max()\
                 .clip(region)
    return  VI_Dataset 

start_time = '06-01'
end_time = '06-30'


ndvi_2018 = MOD13Q_NDVI('2018-'+start_time, '2018-'+end_time) 
ndvi_2019 = MOD13Q_NDVI('2019-'+start_time, '2019-'+end_time)   
ndvi_2020 = MOD13Q_NDVI('2020-'+start_time, '2020-'+end_time)   
ndvi_2021 = MOD13Q_NDVI('2021-'+start_time, '2021-'+end_time)

ndvi_avg = aie.ImageCollection([ndvi_2018,ndvi_2019,ndvi_2020]).mean()
ndvi_dif_forest = (ndvi_2021.subtract(ndvi_avg)).updateMask(forest.eq(aie.Image.constant(1)))


## 数据可视化

In [5]:
ndvi_vis  = {
    'bands': 'NDVI',
    'min': 0,
    'max': 8000,
    'palette': [ 
        '#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718',
        '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201',
        '#004C00', '#023B01', '#012E01', '#011D01', '#011301'
    ]
}

ndvi_dif_vis = {
    'min': -1000,
    'max': 2000,
    'palette': [
        '#D7191C', '#1A9641'
    ]
}

map.addLayer(
    ndvi_avg,
    ndvi_vis,
    'NDVI',
    bounds=region.getBounds()
)

map.addLayer(
    ndvi_dif_forest,
    ndvi_dif_vis,
    'NDVI_dif_forest',
    bounds=region.getBounds()
)
map

Map(center=[29.154125, 120.017469], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [7]:
task = aie.Export.image.toAsset(ndvi_dif_forest, 'NDVI_dif_forest', 2000)   
task.start()

task = aie.Export.image.toAsset(ndvi_avg, 'NDVI', 2000)   
task.start()

task = aie.Export.image.toAsset(img, 'LC_data', 1000)   
task.start()

task = aie.Export.image.toAsset(forest, 'Forest', 1000)   
task.start()

region_feature = aie.FeatureCollection(aie.Feature(region))
task = aie.Export.feature.toAsset(region_feature, 'shapefile')   
task.start()